In [14]:
from pymoo.core.problem import Problem
from pymoo.algorithms.soo.nonconvex.ga import GA
import autograd.numpy as anp

from pymoo.optimize import minimize

import garpar as gp
from garpar.utils.mabc import hparam
from garpar.optimize import OptimizerABC


class _YuEntropy2014Problem(Problem):
    def __init__(
        self, n_var, returns, cov, target_returns, target_var, port_df
    ):

        super().__init__(n_var=n_var, n_obj=1, n_constr=4, type_var=anp.double)

        eps = anp.finfo(anp.double).eps

        self._returns = returns
        self._cov = cov
        self._target_returns = target_returns
        self._target_var = target_var
        self._port_df = port_df

        self.xl = anp.full(n_var, eps, dtype=anp.double)
        self.xu = anp.ones(n_var, dtype=anp.double)

    def _evaluate(self, x, out, *args, **kwargs):

        # sum(wi * ln(wi))
        f = anp.sum(x * anp.log(x), axis=1)

        # sum(ri * wi) >= mu
        g1 = (
            anp.sum(self._returns * x, axis=1) - self._target_returns
        ) / self._target_returns

        # sum(wi) = 1
        g2 = -anp.sum(x, axis=1) + 1
        g3 = -g2

        g4 = anp.zeros(len(x), dtype=x.dtype)
        for idx, weights in enumerate(x):
            newport = gp.Portfolio.from_dfkws(self._port_df, weights=weights)
            g4[idx] = (
                -newport.risk.pf_var() + self._target_var
            ) / self._target_var

        out["F"] = f
        out["G"] = -anp.column_stack([g1, g2, g3, g4])


class YuEntropy2014(OptimizerABC):

    returns = hparam(default="mah")
    returns_kw = hparam(factory=dict)

    covariance = hparam(default="sample_cov")
    covariance_kw = hparam(factory=dict)

    def optimize(self, port, target_returns, target_var):
        n_var = len(port.stocks)
        returns = port.ereturns(self.returns, **self.returns_kw).to_numpy()
        cov = port.covariance(self.covariance, **self.covariance_kw).to_numpy()

        problem = _YuEntropy2014Problem(
            n_var=n_var,
            returns=returns,
            cov=cov,
            target_returns=target_returns,
            target_var=target_var,
            port_df=port._df.copy(),
        )
        weigths = anp.random.random((100,n_var))
        wn = weigths/weigths.sum(axis=1)[True].T

        solver = GA(pop_size=100, sampling=wn)
        res = minimize(problem=problem, algorithm=solver, verbose=True)
        return res


port = gp.datasets.RissoLevyStable(random_state=42).make_portfolio()
res = YuEntropy2014().optimize(port, 0.00000002, 0.00000005)
print(res.X)

n_gen |  n_eval |   cv (min)   |   cv (avg)   |     fopt     |     favg    
    1 |     100 |  1.62757E+01 |  2.14477E+04 |            - |            -
    2 |     200 |  6.510717132 |  6.80170E+03 |            - |            -
    3 |     300 |  6.510717132 |  8.98808E+02 |            - |            -
    4 |     400 |  6.510717132 |  2.56696E+01 |            - |            -
    5 |     500 |  6.510717132 |  1.72207E+01 |            - |            -
    6 |     600 |  4.845111875 |  1.26920E+01 |            - |            -
    7 |     700 |  4.127323915 |  1.02364E+01 |            - |            -
    8 |     800 |  3.403365496 |  8.318852708 |            - |            -
    9 |     900 |  2.779151019 |  6.359909028 |            - |            -
   10 |    1000 |  2.054840106 |  5.057625565 |            - |            -
   11 |    1100 |  1.302416706 |  4.163390995 |            - |            -
   12 |    1200 |  1.302416706 |  3.453249477 |            - |            -
   13 |    1

In [13]:
n_var= 3
weigths = anp.random.random((2,n_var))
wn= weigths/weigths.sum(axis=1)[True].T
wn.sum(axis=1)

array([1., 1.])

In [ ]:
from pymoo.core.problem import Problem
from pymoo.algorithms.soo.nonconvex.ga import GA
import autograd.numpy as anp

from pymoo.optimize import minimize

import garpar as gp
from garpar.utils.mabc import hparam
from garpar.optimize import OptimizerABC


class _YuEntropy2014Problem(Problem):
    def __init__(
        self, n_var, returns, cov, target_returns, target_var, port_df
    ):

        super().__init__(n_var=n_var, n_obj=1, n_constr=4, type_var=anp.double)

        eps = anp.finfo(anp.double).eps

        self._returns = returns
        self._cov = cov
        self._target_returns = target_returns
        self._target_var = target_var
        self._port_df = port_df

        self.xl = anp.full(n_var, eps, dtype=anp.double)
        self.xu = anp.ones(n_var, dtype=anp.double)

    def _evaluate(self, x, out, *args, **kwargs):

        # sum(wi * ln(wi))
        f = anp.sum(x * anp.log(x), axis=1)

        # sum(ri * wi) >= mu
        g1 = (
            anp.sum(self._returns * x, axis=1) - self._target_returns
        ) / self._target_returns

        # sum(wi) = 1
        g2 = -anp.sum(x, axis=1) + 1
        g3 = -g2

        g4 = anp.zeros(len(x), dtype=x.dtype)
        for idx, weights in enumerate(x):
            newport = gp.Portfolio.from_dfkws(self._port_df, weights=weights)
            g4[idx] = (
                -newport.risk.pf_var() + self._target_var
            ) / self._target_var

        out["F"] = f
        out["G"] = -anp.column_stack([g1, g2, g3, g4])


class YuEntropy2014(OptimizerABC):

    returns = hparam(default="mah")
    returns_kw = hparam(factory=dict)

    covariance = hparam(default="sample_cov")
    covariance_kw = hparam(factory=dict)

    def optimize(self, port, target_returns, target_var):
        n_var = len(port.stocks)
        returns = port.ereturns(self.returns, **self.returns_kw).to_numpy()
        cov = port.covariance(self.covariance, **self.covariance_kw).to_numpy()

        problem = _YuEntropy2014Problem(
            n_var=n_var,
            returns=returns,
            cov=cov,
            target_returns=target_returns,
            target_var=target_var,
            port_df=port._df.copy(),
        )
        weigths = anp.random.random((100,n_var))
        wn = weigths/weigths.sum(axis=1)[True].T

        solver = GA(pop_size=100, sampling=wn)
        res = minimize(problem=problem, algorithm=solver, verbose=True)
        return res


port = gp.datasets.RissoLevyStable(random_state=42).make_portfolio()
res = YuEntropy2014().optimize(port, 0.00000002, 0.00000005)
print(res.X)

